In [2]:
from pymongo import MongoClient
import pymongo
from riotwatcher import RiotWatcher

In [50]:
DATA_DB = "data"
GAME_TABLE = "games"
SUMMONER_TABLE = "summoners"
QUEUES = "queues"
GAME_QUEUE = "game_queue"
SUMMONER_QUEUE = "summoner_queue"
QUEUE_NAME = "summoner_queue"
URL = "192.168.5.100"

In [4]:
#key = raw_input("Please enter your Riot key: ")
#target_num = input("Please enter number of games to scrape: ")
#watcher = RiotWatcher(key)

In [5]:
def get_summoner_queue():
    
    return client[DATABASE_NAME][QUEUE_NAME]

In [6]:
test = get_summoner_queue()
print test
print type(test)
print test.name
for x in test.find():
    print x

Collection(Database(MongoClient('192.168.5.100', 27017), u'test'), u'summoner_queue')
<class 'pymongo.collection.Collection'>
summoner_queue


In [16]:
client = MongoClient('mongodb://'+URL)
get_next_in_queue(client[QUEUES], SUMMONER_QUEUE)

{u'lastErrorObject': {u'connectionId': 12, u'err': None, u'n': 1, u'ok': 1.0},
 u'ok': 1.0,
 u'value': {u'_id': ObjectId('556f88175a55a64de122aa7a'), u'id': 23376539}}

In [17]:
game_queue = []
summoner_queue = []
while True:
    if len(summoner_queue) == 0:
        get_summoners_from_queue(10)
    if len(game_queue) == 0:
        get_games_from_summoner(summoner_queue.pop())
    if len(game_queue) > 0:
        scrape_game(game_queue.pop())
        

NameError: name 'get_summoners_from_queue' is not defined

In [100]:
class MongoDBQueue:
    def __init__(self, db, collection, url="127.0.0.1", port="27017"):
        self.url = url
        self.port = port
        self.db = db
        self.collection = collection
        self.connection = MongoClient("mongodb://"+url+":"+port)
        
    def pop(self):
        return self.connection[self.db][self.collection].find_one_and_delete({},None,[("_id",pymongo.ASCENDING)])
        #return self.connection[self.db].command("findandmodify", self.collection, query = {}, sort = {"_id": pymongo.ASCENDING}, remove=True)
        
    def push(self, doc):
        return self.connection[self.db][self.collection].insert(doc)
        #return self.connection[self.db].command("insert", self.collection, doc)
    

In [101]:
summoner_queue = MongoDBQueue(QUEUES, SUMMONER_QUEUE)
game_queue = MongoDBQueue(QUEUES, GAME_QUEUE)

In [103]:
summoner_queue.push({"id":1})
summoner_queue.push({"id":2})
print summoner_queue.pop()

{u'_id': ObjectId('556f97f85a55a64d83c9f008'), u'id': 2}


In [ ]:
class MongoDBReadOnly:
    def __init__(self, db, collection, url="127.0.0.1", port="27017"):
        self.url = url
        self.port = port
        self.db = db
        self.collection = collection
        self.connection = MongoClient("mongodb://"+url+":"+port)
        
    def push(self, doc):
        return self.connection[self.db][self.collection].insert(doc)
    
    def get(self):
        return self.connection[self.db][self.collection].find_one({})


In [68]:
games_table = MongoDBReadOnly(DATA_DB, GAME_TABLE)
summoners_table = MongoDBReadOnly(DATA_DB, SUMMONER_TABLE)

In [69]:
games_table.push({"matchID":13})

ObjectId('556f954b5a55a64d83c9effe')

In [70]:
games_table.get()

{u'_id': ObjectId('556f94b65a55a64d83c9effa'), u'matchID': 13}